# JavaScript pragmatics

## Primitives

### Validation

Very basic validation, rejecting null, undefined, and empty string:

In [48]:
function isTruthy(value) {
  return (!value) ? false : true; 
}

console.log(isTruthy(null));      // => false
console.log(isTruthy(""));        // => false
console.log(isTruthy(undefined)); // => false

console.log(isTruthy(1));         // => true
console.log(isTruthy("foo"));     // => true

false
false
false
true
true


undefined

However, this approach is foiled by the falsiness of zero, which means that we cannot use it to *validate* a numeric value of zero, if we want to do that:

In [11]:
function isTruthy(value) {
  return (!value) ? false : true; 
}

console.log(isTruthy(0));         // => false

false


undefined

A second condition must be added to handle the falsiness of zero:

In [16]:
function isTruthy(value) {
  return (!value || value === 0) ? false : true; 
}

console.log(isTruthy(null));      // => false
console.log(isTruthy(""));        // => false
console.log(isTruthy(undefined)); // => false

console.log(isTruthy(1));         // => true
console.log(isTruthy("foo"));     // => true

console.log(isTruthy(0));         // => false

false
false
false
true
true
false


undefined

### Generating id numbers using `Date()`

In [1]:
var id = (new Date()).getTime();
console.log(id);

1434946515996


undefined

## Variables

Test to see if a variable is defined, and if it’s undefined, set it:

In [1]:
var myString = "foo";
myString = myString || "bar";

console.log(myString); // => "foo"

foo


undefined

In [2]:
var myString = "";
myString = myString || "bar";

console.log(myString); // => "bar"

bar


undefined

Use the above to test for omitted optional function parameters (see [JavaScript Function Parameters](http://www.w3schools.com/js/js_function_parameters.asp)):

In [3]:
function myFunction(x, y) {
    y = y || 0;
    return [x, y];
}
console.log(myFunction(1));    // => [1, 0]
console.log(myFunction(1, 2)); // => [1, 2]

[ 1, 0 ]
[ 1, 2 ]


undefined

The above approaches are better than using an expression like `if (!value) ...`, because of the falsiness of zero. Here, that expression works as intended, assigning the value `1` to the omitted function parameter `y`:

In [4]:
function myFunction(x, y) {
    if (!y) y = 1;
    console.log(x, y);
}

myFunction(0); // => "0 1"

0 1


undefined

Here, however, it does not work as intended. Because zero is falsey, `!0` returns `true`, and we cannot set the `y` parameter to the value `0`:

In [10]:
if (0) console.log('Zero is truthy');  // => ""
if (!0) console.log('Zero is falsey'); // => "Zero is falsey"

function myFunction(x, y) {
    if (!y) y = 1;                     // `(!0)` will return `true` here
    console.log(x, y);
}

// We want to set `y` to `0`, but it gets set to `1`
myFunction(1, 0); // => "1 1"

Zero is falsey
1 1


undefined

## Strings

Replace (here, leading and trailing spaces) using regexp:

In [5]:
var myString = "   I'm a string   ";

// => "I'm a string without leading or trailing spaces"
console.log(myString.replace(/^\s+|\s+$/g, "") +
    " without leading or trailing spaces");

I'm a string without leading or trailing spaces


undefined

Test for presence of a substring using `indexOf()`:

In [6]:
var myString = "foobar";
console.log(myString.indexOf("foo") !== -1); // => true

true


undefined

## Arrays

Test for presence of element of an array using `indexOf()`:

In [7]:
console.log([1, 2, 3].indexOf(1) !== -1);         // => true
console.log(["foo", "bar"].indexOf("foo") !== -1) // => true

true
true


undefined

Use the `.map()` method instead of a `for` loop to transform an array:

In [8]:
var myArray = [0, 1, 2, 3, 4];
var myArrayTransformed = myArray.map(function(arrayElement) {
        return arrayElement * 2;
    });
console.log(myArrayTransformed); // => [ 0, 2, 4, 6, 8 ]

[ 0, 2, 4, 6, 8 ]


undefined

## Objects

Extend obj1 with properties of obj2:

In [9]:
var obj1 = {
    abstraction: "high"
};

var obj2 = {
    abstraction: "middling",
    concretion:  "low"
};

function extend(obj1, obj2) {
    for (var prop in obj2) {

        // Skip properties already belonging to obj1
        if (prop in obj1) {
            continue;
        } else {
            obj1[prop] = obj2[prop];
        }
    }
}

extend(obj1, obj2);
console.log(obj1);

{ abstraction: 'high', concretion: 'low' }


undefined

## Functions

### Recursion

Strips leading space, invoking itself recursively:

In [10]:
function stripLeadingSpace(input) {
    while (input.charAt(0) == " ") {
        input = input.substring(1, input.length);
        stripLeadingSpace(input);
    }
    return input;
}

// => "[foo   ]"
console.log("[" + stripLeadingSpace("   foo   ") +"]");

[foo   ]


undefined

Strips trailing space, invoking itself recursively:

In [11]:
function stripTrailingSpace(input) {
    while (input.charAt(input.length - 1) == " ") {
        input = input.substring(0, input.length - 1);
        stripTrailingSpace(input);
    }
    return input;
}

// => "[   bar]"
console.log("[" + stripTrailingSpace("   bar   ") + "]");

[   bar]


undefined

## DOM manipulation

* Instead of using `window.onload = …`, which can be overwritten by use of the same event handler elsewhere, use `window.AddEventListener('load', function() {…});` or `window.AddEventListener('DOMContentLoaded', function() {…});`.
* Use a document fragment (`var frag = document.createDocumentFragment();`) for efficiency in DOM manipulation: append all children to the fragment, then append the fragment to an existing DOM element. This means only a single manipulation of the DOM, rather than many.

### Get and validate form input data

In [ ]:
// Object constructor function passed to function below.
function Constructor() {
    
    // Include an `id` property generated from the current time in seconds.
    this.id = Math.floor(Date.now() / 1000);
}


/**
 * Returns an object including selected 'id' and 'value' attributes
 * of form input elements. Optionally, calls a validator function
 * and an object constructor function.
 *
 * @param  {string} formId         'id' attribute of a form element
 * @param  {array} inputIds        'id' attributes of desired form input elements
 * @param  {function} validator    Function to validate form input element values
 * @param  {function} Constructor  Object constructor function
 * @return {object}
 */
function getFormInputData(formId, inputIds, validator, Constructor) {

    /* Get input elements of the form element with the 'id' attribute
    provided as the `formId` parameter, and create a new empty object
    using the `Constructor` parameter. */
    var formInputData = document.forms[formId].getElementsByTagName('input'),
        myObject = new Constructor();

    /* Iterate over `formInputData` selecting those items whose 'id' attributes
    appear in the `inputIds` array, adding their 'id' attributes as properties
    of the object `myObject` and their 'value' attributes as property values. */
    for (var i = 0, l = formInputData.length; i < l; i++) {
        var formInputDataItemId  = formInputData[i].id,
            formInputDataItemVal = formInputData[i].value;

        if (inputIds.indexOf(formInputDataItemId) !== -1) {
            if (!validator(formInputDataItemVal)) return false;
            myObject[formInputDataItemId] = formInputDataItemVal;
        }
    }

    return myObject;
}